In [90]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from binance.client import Client
import configparser

In [91]:
btc_tweets = pd.read_csv('BTC_tweets.csv')
eth_tweets = pd.read_csv('ETH_tweets.csv')
sol_tweets = pd.read_csv('SOL_tweets.csv')
doge_tweets = pd.read_csv('DOGE_tweets.csv')

In [92]:
btc_tweets

,Unnamed: 0,Date,User,Content,Followers,Friends,Statuses
0,0,2017-09-13,BMouler,Mike Krieger Asks Which Is 'Fraudulent': Bitco...,6922,586,69073
1,1,2017-09-13,big_blockers,The current price of Bitcoin is $3941.87.\n Th...,3041,165,107635
2,2,2017-09-13,theperson3233,@officialmcafee You talking about Bitcoin cash...,397,610,4528
3,3,2017-09-13,DisruptBanksy,Buy or sell $100 of #digitalcurrency or more v...,4490,4974,148694
4,4,2017-09-13,JasperJove,"@officialmcafee Hi John, when you say Bitcoin ...",182,555,7548
...,...,...,...,...,...,...,...
16507,16507,2022-11-09,big_blockers,The current price of Bitcoin is $18458.95.\nTh...,3041,165,107635
16508,16508,2022-11-09,japaneur,Think about all the problems Japan could solve...,357,520,5749
16509,16509,2022-11-09,crypt0potamus,@reno59392950 @AdithiaKusno @LynAldenContact B...,492,1253,4345
16510,16510,2022-11-09,CStackamoto,@CryptoVinco Who cares what anyone says about ...,392,248,6734


In [93]:
import re

In [94]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub("@[\w]*", "", text)
    text = re.sub("http\S+", "", text)
    text = re.sub("[^a-zA-z#]", " ", text)
    text = re.sub("rt", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [95]:
btc_tweets['Content'] = btc_tweets['Content'].apply(preprocess_text)

In [96]:
btc_tweets

,Unnamed: 0,Date,User,Content,Followers,Friends,Statuses
0,0,2017-09-13,BMouler,mike krieger asks which is fraudulent bitcoin ...,6922,586,69073
1,1,2017-09-13,big_blockers,the current price of bitcoin is the current pr...,3041,165,107635
2,2,2017-09-13,theperson3233,you talking about bitcoin cash or btc,397,610,4528
3,3,2017-09-13,DisruptBanksy,buy or sell of #digitalcurrency or more via yo...,4490,4974,148694
4,4,2017-09-13,JasperJove,hi john when you say bitcoin you mean btc or ...,182,555,7548
...,...,...,...,...,...,...,...
16507,16507,2022-11-09,big_blockers,the current price of bitcoin is the current pr...,3041,165,107635
16508,16508,2022-11-09,japaneur,think about all the problems japan could solve...,357,520,5749
16509,16509,2022-11-09,crypt0potamus,bitcoin is backed by pure energy and math so ...,492,1253,4345
16510,16510,2022-11-09,CStackamoto,who cares what anyone says about price predic...,392,248,6734


In [97]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [98]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [99]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [100]:

result = model(tokens)

In [101]:

result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [102]:

result.logits
int(torch.argmax(result.logits))+1

4

In [103]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [33]:
def get_sentiment(row):
    score = sentiment_score(row)
    if score>3:
        return "Positive"
    elif score >=:
        return "Neutral"
    else:
        return "Negative"

In [68]:
btc_tweets.iloc[16510]['Content']

' who cares what anyone says about price predictions nobody can predict the exact price of any commodity at a specific time other than the long term price of btc is clearly upward i ll tell you who doesn t give a fuck about what bears or maxis say bitcoin doesn t give a fuck '

In [51]:
test = "so are you on the side if #btc with the capitalistic and selfish majority thus for #democracy or are you on the side of the creator of bitcoin and for electronic independent consensus controlled cash meaning #bch "

In [52]:
get_sentiment(test)

'Negative'

In [104]:
btc_tweets['Sentiment'] = btc_tweets['Content'].apply(sentiment_score)

In [105]:
eth_tweets['Sentiment'] = eth_tweets['Content'].apply(sentiment_score)

In [106]:
sol_tweets['Sentiment'] = sol_tweets['Content'].apply(sentiment_score)
doge_tweets['Sentiment'] = doge_tweets['Content'].apply(sentiment_score)

In [114]:
btc_tweets['Sentiment'].value_counts()

1    10933
3     2093
5     2012
2      953
4      521
Name: Sentiment, dtype: int64

In [120]:
btc_tweets.to_csv('BTC_tweets_with_sentiment.csv')
eth_tweets.to_csv('ETH_tweets_with_sentiment.csv')
doge_tweets.to_csv('Doge_tweets_with_sentiment.csv')

In [115]:
eth_tweets['Sentiment'].value_counts()

1    3522
5     938
2     473
3     459
4     407
Name: Sentiment, dtype: int64

In [116]:
sol_tweets['Sentiment'].value_counts()

5    3
1    1
Name: Sentiment, dtype: int64

In [119]:
doge_tweets['Sentiment'].value_counts()

1    266
5     57
3     26
2     16
4     10
Name: Sentiment, dtype: int64

In [113]:
btc_tweets[btc_tweets['Sentiment']==5]

,Unnamed: 0,Date,User,Content,Followers,Friends,Statuses,Sentiment
3,3,2017-09-13,DisruptBanksy,buy or sell of #digitalcurrency or more via yo...,4490,4974,148694,5
36,36,2017-09-13,Teokanistras,sta mining effectively with your computer or s...,141056,114314,856105,5
46,46,2017-09-13,sharedkran,bitcoin generator online btc instant lets see ...,34,447,1340,5
53,53,2017-09-13,CRYPTOCURRBIT,earn interest on lending litecoin or ethereum ...,2754,4959,44532,5
57,57,2017-09-13,lakeemwilliams,bitcoin btc direct to your bitcoin wallet wit...,3311,1934,122548,5
...,...,...,...,...,...,...,...,...
16462,16462,2022-11-09,MarketNomad_,it s amazing how people always wish they bough...,134,330,502,5
16464,16464,2022-11-09,CryptoMoonMag,the real power of #bitcoin is that you don t ...,11730,29,446,5
16468,16468,2022-11-09,CannabisNFTMag,get in #bitcoin now at blockfi why just hodl w...,461,72,8952,5
16481,16481,2022-11-09,boomer_btc,the development of these cards is a great exa...,3883,812,4797,5
